# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import pickle
import re
import sys
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.tree import DecisionTreeClassifier
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load data from database
def load_data():
    '''
    Load data, transform DataFrame, get X, Y and name of feature columns for score results
    
    INPUT:
    engine - create data base
    df - read a table of engine
    
    OUTPUT:
    X - an array with columns messages from df
    Y - a new dataframe that has the following characteristics:
            1. has all comns except for 'id', 'message', 'original', and 'genre'.
            2. has a column 'related' cleaned from values 2.
    categiries - a list of columns names in Y.    
    '''
    
    engine = create_engine('sqlite:///messages.db')
    df = pd.read_sql_table("messages", engine)  
    X = df['message'].values
    Y = df.drop(['id', 'message', 'original', 'genre'],axis=1)
    Y['related'] = Y['related'].map(lambda x: 1 if x == 2 else x)
    categories = Y.columns
    
    return X, Y, categories

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    '''
    Clean, normalize, tokenize, lemmatize a text
    
    INPUT:
    text - a string, in this case messages
    
    OUTPUT:
    clean_tokens - an array with a text that has the following characteristics:
            1. has no punctuation marks
            2. splited into sequence of words
            3. cleaned from stopwords
            4. lemmatized
            5. all letters are in low case
    '''
    
    #normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize
    tokens = word_tokenize(text)
    
    #stop_words
    my_stopwords=stopwords.words('english')
    tokens = [word for word in tokens if word not in my_stopwords]
    
    #lemmatization
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def simple_model():
    '''
    Create a pipeline for the Machine Learning
    
    OUTPUT:
    pipeline that has:
            1. transform estimator with tokenize function
            2. transform estimator TF IDF
            3. classifier multi output estimator
    '''
    
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
    ]) 
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
def main_simple():
      
    '''
    Main function that:
        1. loads datas and gets X, Y und categories
        2. splits the data into training and test 
        3. defines a model - simple_model
        4. fits the model
        5. predicts on test data
        6. prints results for each feature
    
    '''
        
    X, Y, categories = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    # train classifier
    model = simple_model()
    model.fit(X_train, y_train)
    
    # predict on test data
    y_pred = model.predict(X_test)
    
    #score report
    y_pred_pd = pd.DataFrame(y_pred, columns = categories)
    for column in categories:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column],y_pred_pd[column]))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
#see results
main_simple()

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.60      0.54      0.57      1512
           1       0.87      0.89      0.88      5042

    accuracy                           0.81      6554
   macro avg       0.74      0.72      0.73      6554
weighted avg       0.81      0.81      0.81      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.90      0.97      0.93      5434
           1       0.77      0.48      0.59      1120

    accuracy                           0.89      6554
   macro avg       0.84      0.73      0.76      6554
weighted avg       0.88      0.89      0.88      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1     

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6197
           1       0.75      0.05      0.09       357

    accuracy                           0.95      6554
   macro avg       0.85      0.52      0.53      6554
weighted avg       0.94      0.95      0.93      6554

------------------------------------------------------

FEATURE: electricity

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6428
           1       0.58      0.06      0.10       126

    accuracy                           0.98      6554
   macro avg       0.78      0.53      0.55      6554
weighted avg       0.97      0.98      0.97      6554

------------------------------------------------------

FEATURE: tools

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        43

    accuracy                    

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
def main_grid():
    
    '''
    Main function for grid search that:
        1. loads datas and gets X, Y und categories
        2. splits the data into training and test 
        3. defines a pipeline as a model - simple_model pipeline
        4. sets parameters for an estimator
        5. defines grid search model with the pipeline and parameters
        6. fits the model
        7. predicts on test data
        8. prints results for each feature
    
    '''
    X, Y, categories = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    model = simple_model()
    
    #use the same pipeline
    #add parameters
    parameters = {
        'clf__estimator__n_estimators': [20, 50]
    }
    
    cv = GridSearchCV(estimator=model, param_grid=parameters, cv=3, scoring='f1_weighted', verbose=3)
    
    #fitting
    cv.fit(X_train, y_train)
    
    #predict
    y_pred = cv.predict(X_test)
    
    #score report
    y_pred_pd = pd.DataFrame(y_pred, columns = categories)
    for column in categories:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column],y_pred_pd[column]))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
#see results
main_grid()

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... clf__estimator__n_estimators=20, score=0.535, total= 1.1min
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV] ..... clf__estimator__n_estimators=20, score=0.538, total= 1.2min
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.3min remaining:    0.0s


[CV] ..... clf__estimator__n_estimators=20, score=0.540, total= 1.2min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.549, total= 2.5min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.554, total= 2.4min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.544, total= 2.6min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 11.0min finished


------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.71      0.45      0.55      1570
           1       0.84      0.94      0.89      4984

    accuracy                           0.82      6554
   macro avg       0.78      0.70      0.72      6554
weighted avg       0.81      0.82      0.81      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5483
           1       0.81      0.52      0.64      1071

    accuracy                           0.90      6554
   macro avg       0.86      0.75      0.79      6554
weighted avg       0.90      0.90      0.89      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1     

              precision    recall  f1-score   support

           0       0.88      0.97      0.92      5344
           1       0.75      0.43      0.55      1210

    accuracy                           0.87      6554
   macro avg       0.82      0.70      0.74      6554
weighted avg       0.86      0.87      0.85      6554



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
def improved_model():
    
    '''
    Create a pipeline and parameters for a grid search model
    
    OUTPUT:
    cv - model that:
            1. defines an improved pipeline 
            2. sets parameters for estimators
            3. defins a grid search model with the pipeline and parameters
    '''
    
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(
            AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
        ))
    ])

    parameters = {
        'clf__estimator__learning_rate': [0.1, 0.3],
        'clf__estimator__n_estimators': [100, 200]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

In [16]:
#to fix pickle saving issue

model = improved_model()


In [17]:
def main():
    
    '''
    Main function for grid search that:
        1. loads datas and gets X, Y und categories
        2. splits the data into training and test 
        3. defines a model - improved_model
        4. fits the model
        5. predicts on test data
        6. prints results for each feature
    
    '''
    
    X, Y, categories = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    #score report
    y_pred_pd = pd.DataFrame(y_pred, columns = categories)
    for column in categories:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column],y_pred_pd[column]))

In [18]:
#see results
main()

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.46      0.86      0.60      1542
           1       0.94      0.69      0.80      5012

    accuracy                           0.73      6554
   macro avg       0.70      0.77      0.70      6554
weighted avg       0.83      0.73      0.75      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.94      0.87      0.91      5412
           1       0.55      0.75      0.64      1142

    accuracy                           0.85      6554
   macro avg       0.75      0.81      0.77      6554
weighted avg       0.88      0.85      0.86      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      6524
           1     

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5928
           1       0.84      0.85      0.84       626

    accuracy                           0.97      6554
   macro avg       0.91      0.91      0.91      6554
weighted avg       0.97      0.97      0.97      6554

------------------------------------------------------

FEATURE: cold

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      6433
           1       0.30      0.72      0.43       121

    accuracy                           0.96      6554
   macro avg       0.65      0.84      0.70      6554
weighted avg       0.98      0.96      0.97      6554

------------------------------------------------------

FEATURE: other_weather

              precision    recall  f1-score   support

           0       0.98      0.88      0.92      6228
           1       0.20      0.59      0.30       326

    accuracy                   

### 9. Export your model as a pickle file

In [19]:
pickle.dump(model, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.